In [3]:
from lib.main_func_p1 import timer, path
import pandas as pd
from importlib import reload
from collections import OrderedDict
import lib.main_func_p4
from lib.main_func_p4 import modelXGBoost_fit_scores
import os

#XGBoost library
import xgboost as xgb

In [4]:
#####################################
# proteina (uniprot_ID)
uniprot_id = 'P22303'
path_file = path(uniprot_id)
# Parametros
seed = 142854
fp_name = 'morgan2_c'

# Lista de resultados BayesSearchCV
results_list =  os.listdir(f'./top_scores/{uniprot_id}')
results_list

['P22303_20220621111120_top_scores_XGBClassifier_accuracy_rf0.xlsx',
 'P22303_20220621114435_top_scores_XGBClassifier_precision_rf0.xlsx',
 'P22303_20220621124638_top_scores_XGBClassifier_balanced_accuracy_rf0.xlsx',
 'P22303_20220621133849_top_scores_XGBClassifier_jaccard_rf0.xlsx',
 'P22303_20220621140847_top_scores_XGBClassifier_f1_weighted_rf0.xlsx']

In [23]:
# Load train and validation datasets
df_train = pd.read_pickle(f'{path_file}_dataset_train')
df_valid = pd.read_pickle(f'{path_file}_dataset_valid')

# Cargar archivo / eliminar columnas innecesarias
for name in results_list:

    excel_name = f'./top_scores/{uniprot_id}/{name}'
    metric_score = '_'.join(str.split(excel_name,'_')[6:-1])
    print(excel_name)
    print(f'model_{metric_score}')

    result_len = 9
    resample_factor = int(str.split(excel_name,'_')[-1][2:-5])
    resample_mode = 'under_sampling'

    df_ori = pd.read_excel(excel_name, sheet_name=0)
    df_top_results = df_ori[['params_dict', 'AUC_train', 'AUC_valid']]
    df_top_results = df_top_results.drop_duplicates()
    df_top_results.head(5)

    for i, params_dict in enumerate(df_top_results.params_dict.iloc[0:result_len]):
        eval_metric = ['error', 'auc']
        tick = timer()
        params_dict = dict(eval(params_dict))
        default_params_xgb = {'booster': 'gbtree', 'tree_method': 'gpu_hist',
                              'objective':'binary:logistic', 'grow_policy': 'depthwise',
                              'eval_metric': eval_metric, 'early_stopping_rounds':10}
        params_dict.update(default_params_xgb)


        xgb_clf = xgb.XGBClassifier(**params_dict)

        # Train model and evaluating scores (train / validation)
        xgb_clf, scores_train, scores_valid = modelXGBoost_fit_scores(xgb_clf, fp_name, df_train, df_valid,
                                                                      resample_factor=resample_factor,
                                                                      resample_mode=resample_mode)

        xgb_clf.save_model(f'./models_test/{uniprot_id}/model_top_{metric_score}_{i+1}.ubj')

        print(f'{i+1}/{result_len}. AUC_score=(train={scores_train[0]},'
              f' valid={scores_valid[0]}). Time elapsed: {timer(tick)}')



./top_scores/P22303/P22303_20220621111120_top_scores_XGBClassifier_accuracy_rf0.xlsx
model_accuracy
1/9. AUC_score=(train=0.996, valid=0.965). Time elapsed: 3.9 seconds.
2/9. AUC_score=(train=0.995, valid=0.96). Time elapsed: 3.2 seconds.
3/9. AUC_score=(train=0.994, valid=0.961). Time elapsed: 3.2 seconds.
4/9. AUC_score=(train=0.995, valid=0.964). Time elapsed: 3.0 seconds.
5/9. AUC_score=(train=0.995, valid=0.963). Time elapsed: 3.3 seconds.
6/9. AUC_score=(train=0.994, valid=0.96). Time elapsed: 3.4 seconds.
7/9. AUC_score=(train=0.994, valid=0.96). Time elapsed: 3.4 seconds.
8/9. AUC_score=(train=0.995, valid=0.961). Time elapsed: 3.2 seconds.
9/9. AUC_score=(train=0.995, valid=0.961). Time elapsed: 3.6 seconds.
./top_scores/P22303/P22303_20220621114435_top_scores_XGBClassifier_precision_rf0.xlsx
model_precision
1/9. AUC_score=(train=0.994, valid=0.961). Time elapsed: 5.2 seconds.
2/9. AUC_score=(train=0.994, valid=0.962). Time elapsed: 5.0 seconds.
3/9. AUC_score=(train=0.994, va